In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### We are going to import our cleaned dataset, From my Data Preprocessing Project

In [6]:
df = pd.read_csv("../DATA/AMES_Final_DF.csv")

In [7]:
df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,441.0,...,0,0,0,0,1,0,0,0,1,0
1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,270.0,...,0,0,0,0,1,0,0,0,1,0
2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,406.0,...,0,0,0,0,1,0,0,0,1,0
3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,1045.0,...,0,0,0,0,1,0,0,0,1,0
4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,137.0,...,0,0,0,0,1,0,0,0,1,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 274 entries, Lot Frontage to Sale Condition_Partial
dtypes: float64(11), int64(263)
memory usage: 6.1 MB


In [9]:
X = df.drop("SalePrice", axis=1)

In [10]:
y = df["SalePrice"]

In [11]:
from sklearn.model_selection import train_test_split

#### I am holding out 10% of the data to test  the model on a data it's never saw

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=50)

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
scaler = StandardScaler()  # scales the training data from 0 to 1

In [15]:
scaled_X_train = scaler.fit_transform(X_train)

In [16]:
scaled_X_test = scaler.transform(X_test)

#### We are not fitting the the test data because we don't want the model to know the test data

#### We are creating an instance of the ElasticNet model, this will help the model in reducing overfitting and underfitting on the training data


In [17]:
from sklearn.linear_model import ElasticNet

* Creating an Instance of the ElasticNet model

In [18]:
base_elastic_model = ElasticNet(max_iter=1000000)

In [19]:
param_grid = {'alpha':[0.1,1,5,10,100],'l1_ratio':[.1,.7,.99,1]}

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
grid_model = GridSearchCV(base_elastic_model,param_grid=param_grid,scoring='neg_mean_squared_error',
                         cv=5,verbose=1)

In [22]:
grid_model.fit(scaled_X_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5, estimator=ElasticNet(max_iter=1000000),
             param_grid={'alpha': [0.1, 1, 5, 10, 100],
                         'l1_ratio': [0.1, 0.7, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

#### Let's See the best hyperparameter combination

In [23]:
grid_model.best_estimator_

ElasticNet(alpha=100, l1_ratio=1, max_iter=1000000)

#### Then we are to evaluate our model's performance on the 10% per

In [24]:
y_pred = grid_model.predict(scaled_X_test)

In [25]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [26]:
mean_absolute_error(y_test,y_pred)

14726.898588147418

In [27]:
np.sqrt(mean_squared_error(y_test,y_pred))

21690.500001787816

#### How do we know if this is actually are good error value for our predictions

In [28]:
np.mean(df['SalePrice'])

180815.53743589742

#### * This is about 11 percent error in the model predictions

Saving The model

In [43]:
import joblib

In [44]:
joblib.dump(grid_model,"house_price_prediction.joblib")

['house_price_prediction.joblib']